# Exploring Role Skill Expression in League

After reading the [Oracle's Elixir blog post](https://oracleselixir.com/2020/02/luck-and-league-measuring-ideal-season-length-in-lol/) I wanted to apply the same approach to roles in League of Legends. I recalculate the results of the blog post, update the half skill half luck breakpoint with the addition of 2020 data, and then apply the approach to roles in a game.

In [101]:
import math
import pandas as pd
pd.options.display.max_rows = 100
pd.options.display.max_columns = 500
import seaborn as sns

In [102]:
games_2018 = pd.read_csv("../data/2018_LoL_esports_match_data_from_OraclesElixir_20201019.csv")
games_2019 = pd.read_csv("../data/2019_LoL_esports_match_data_from_OraclesElixir_20201019.csv")
games_2020 = pd.read_csv("../data/2020_LoL_esports_match_data_from_OraclesElixir_20201019.csv")

In [103]:
games_2018 = games_2018.replace('NA LCS', 'LCS')

### Validate Function

Recalculating the results from the work of the original [Oracle's Elixir blog post](https://oracleselixir.com/2020/02/luck-and-league-measuring-ideal-season-length-in-lol/) to ensure that I have my math down right.

In [104]:
df = games_2018.append(games_2019, ignore_index=True)

In [105]:
df.shape

(148442, 105)

In [106]:
df = df[df['league'].isin(['LCS', 'LEC'])]
df['datacompleteness'].value_counts()

complete    8292
partial       24
Name: datacompleteness, dtype: int64

In [107]:
df = df[df['datacompleteness'] == 'complete']
df = df[df['playoffs'] == 0]
df.shape

(6624, 105)

In [108]:
#df.head(10)

In [109]:
validation_df = df.groupby(['team', 'year', 'split']).mean().reset_index()

In [110]:
validation_df.shape

(60, 91)

In [111]:
#validation_df.head()

In [112]:
win_percentage_standard_deviation = validation_df['result'].std()
print(win_percentage_standard_deviation)

0.17078795942724667


In [113]:
games_in_split = 18
random_standard_deviation = math.sqrt(.25 / games_in_split)
print(random_standard_deviation)

0.11785113019775792


In [114]:
#var_observed = var_true + var_random
var_observed = win_percentage_standard_deviation ** 2
var_random = random_standard_deviation ** 2
var_true = var_observed - var_random
print("var_observed: ", var_observed)
print("var_random: ", var_random)
print("var_true: ", var_true)

var_observed:  0.029168527085322856
var_random:  0.013888888888888888
var_true:  0.015279638196433968


In [115]:
games_breakpoint = .25 / (var_true)
print(games_breakpoint)

16.361643959498082


In [116]:
def get_games_breakpoint(win_pct_column, num_games_season):
    win_percentage_standard_deviation = win_pct_column.std()
    random_standard_deviation = math.sqrt(.25 / num_games_season)
    var_observed = win_percentage_standard_deviation ** 2
    var_random = random_standard_deviation ** 2
    var_true = var_observed - var_random
    games_breakpoint = .25 / (var_true)
    return games_breakpoint

In [117]:
breakpoint = get_games_breakpoint(validation_df['result'], 18)
print(breakpoint)

16.361643959498082


### Run Function with 2020 Data

In [118]:
games = games_2018.append(games_2019, ignore_index=True)
games = games.append(games_2020, ignore_index=True)
games.shape

(218294, 105)

In [119]:
df = games[games['league'].isin(['LCS', 'LEC'])]
df['datacompleteness'].value_counts()

complete    14280
partial        96
Name: datacompleteness, dtype: int64

In [120]:
df = df[df['datacompleteness'] == 'complete']
df = df[df['playoffs'] == 0]
df.shape

(10992, 105)

In [121]:
#df.head()

In [122]:
season_length_breakpoint_df = df.groupby(['team', 'year', 'split']).mean().reset_index()
#season_length_breakpoint_df.head(10)

In [123]:
breakpoint = get_games_breakpoint(season_length_breakpoint_df['result'], 18)
print(breakpoint)

14.365126087736414


### Analyze Roles

In [138]:
position_labels = ['top', 'jng', 'mid', 'bot', 'sup']
positional_metrics = ['golddiffat10', 'xpdiffat10', 'csdiffat10', 'golddiffat15', 'xpdiffat15', 'csdiffat15']

In [139]:
def determine_positional_win(metric):
    if metric > 0:
        return 1
    else:
        return 0

In [140]:
#position = 'top'
#selected_metric = 'xpdiffat10'

for position in position_labels:
    for selected_metric in positional_metrics:
        roles_df = df[df['position'] == position]
        roles_df['positional_win'] = df[selected_metric].apply(lambda x: determine_positional_win(x))
        role_breakpoint_df = roles_df.groupby(['team', 'year', 'split']).mean().reset_index()
        breakpoint = get_games_breakpoint(role_breakpoint_df['positional_win'], 18)
        print(position, '-', selected_metric, ':',breakpoint)

top - golddiffat10 : 33.289628804063305
top - xpdiffat10 : 45.81466255531417
top - csdiffat10 : 19.997317312119584
top - golddiffat15 : 29.349738310161804
top - xpdiffat15 : 40.57169955965806
top - csdiffat15 : 19.031398719608713
jng - golddiffat10 : 83.6290475698082
jng - xpdiffat10 : 36.93821751615051
jng - csdiffat10 : 66.49571153186001
jng - golddiffat15 : 29.800131564588874
jng - xpdiffat15 : 24.515315927888413
jng - csdiffat15 : 22.81974157625906
mid - golddiffat10 : 32.853965176187025
mid - xpdiffat10 : 33.93739980626277
mid - csdiffat10 : 17.288602354684762
mid - golddiffat15 : 23.265795190143745
mid - xpdiffat15 : 23.166419748516475
mid - csdiffat15 : 15.433406290882287
bot - golddiffat10 : 32.806895767743
bot - xpdiffat10 : 98.97986251112019
bot - csdiffat10 : 28.902612649980117
bot - golddiffat15 : 26.823228458611297
bot - xpdiffat15 : 36.62868154596016
bot - csdiffat15 : 21.196622106067053
sup - golddiffat10 : 77.78750608160144
sup - xpdiffat10 : 69.42575868618843
sup - csd